In [1]:
from lxml import etree
from unicodedata import normalize

In [2]:
parser = etree.HTMLParser()
root = './pages/pag'

In [3]:
# run into the element recursively getting the text
def get_element_text(elem):
    if len(elem) == 0:
        if elem != []:
            if elem.tag == "br":
                return "\n"
            elif elem.text == None:
                return ""
            else:
                return elem.text
        else:
            return ""
    else:
        if elem[0].tag == 'br':
            return "\n" + ("" if elem[0].tail == None else elem[0].tail.strip()) + get_element_text(elem[1:])
        if elem[0].tail != None and elem[0].tail.strip() != "":
            return (elem.text.strip() if isinstance(elem, etree._Element) and elem.text != None else "") + ("" if elem[0].text == None else elem[0].text) + ("" if len(elem[0]) == 0 else get_element_text(elem[0][:])) + elem[0].tail + get_element_text(elem[1:])
        else:
            return (elem.text.strip() if isinstance(elem, etree._Element) and elem.text != None else "") +  ("" if elem[0].text == None else elem[0].text) + ("" if len(elem[0]) == 0 else get_element_text(elem[0][:])) + get_element_text(elem[1:])

In [4]:
def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [5]:
def text_normalize(text):
    return normalize("NFKD",text)

In [29]:
cast_check_list = ["Intérpretes", "Elenco"]
director_check_list = ["Direção", "Diretor"]
script_check_list = ["Roteiro"]
genre_check_list = ["Gênero"]
def get_all_information(elem, depth):
    if depth < 3:
        cast = any(cast in get_element_text(elem) for cast in cast_check_list)
        director = any(direc in get_element_text(elem) for direc in director_check_list)
        script = any(sct in get_element_text(elem) for sct in script_check_list)
        genre = any(gen in get_element_text(elem) for gen in genre_check_list)
        
        if (cast and (director or script)) or (director and genre) or (director and script):
            return ("Found!", elem, depth, get_element_text(elem))
        else:
            return get_all_information(elem.getparent(), depth+1)
    else:
         return ("Not found!", elem, depth)

In [7]:
# todas as possíveis categorias mapeadas para os domínios (forma de categorizar a informação extraída mais facil)
top_words = ("Direção","Elenco","Intérpretes","Gênero","Duração","Classificação","Roteiro","Ano","Produção","Fotografia","Trilha Sonora","Estudio","Montador","Montagem","Distribuidora","Música","Design de Produção","Figurino","Censura")
# todas as palavras que estão na extração, mas não servem e podem atrapalhar a categorização das informações
bad_words = ["Ficha técnica","Equipe técnica"]

def which_top_words_startswith(text):
    if text.startswith(top_words):
        for top in top_words:
            if text.startswith(top):
                return top
    else:
        return None

In [25]:
def extract_by_breakline(element,i):
    info = get_element_text(element).split("\n")
    filme = {}
    filme['id'] = i
    for item in info:
        categoria = which_top_words_startswith(item)
        if categoria != None:
            item = item[len(categoria):].strip() # apenas a informação
            if item.startswith(":"):
                item = item[1:].strip() # apenas a informação, sem ':'
                if item: # não vazio...
                    filme[remover_acentos(categoria).lower()] = text_normalize(item)
    return filme

In [9]:
# breve descrição da lógica proposta para o extrator geral:
# > Se houver categoria e ':' e algum texto depois disso, é uma informação a ser extraída [OK]
# 	> Considera o termo antes de ':' como a categoria, e o que houver depois de ':' a informação

# > Se for uma categoria e não tiver ":" mas tem texto a seguir [OK]
# 	> considera a categoria, remove-a do texto -> strip() -> e atribui o restante como informação

# > Se for uma categoria e não tiver ":" nem texto a seguir, a informação referente pode estar por vir...
# 	> Enquanto não aparecer uma linha em branco, ou uma nova categoria, ou uma stopword, adicionar à ultima categoria vista

# > Se for uma linha vazia, pode considerar que uma nova informação está por vir...
# 	> ignora...

# > Se for uma categoria e tiver ":" mas não houver texto a seguir OU se for uma categoria, não tiver ":" e a próxima linha for em branco, a forma de extração está errada para esse caso
# 	> Chamar o método get_element_text para o div pai, dar split em '\n' e realizar as condições acima

In [80]:
# categorias obrigatórias para seleção de elemento da DOM
list_word = ["intérpretes", "elenco","diretor","direção"]
# 4-tupla para a função startswith
search_words = ("intérpretes:", "elenco:","diretor:","direção:")
# para um caso em específico onde a informação está contida em um parágrafo
second_effort = ("Intérpretes:", "Elenco:", "Diretor:", "Direção:", "Roteiro:")

for i in range(1,101):
    tree = etree.parse(root + str(i) + '.html', parser)
    check = False
    content = None
    # percorre cada elemento da arvore DOM
    for item in tree.iter():
        # busca o elemento que tenha uma das search_words/list_words
        if item.text != None and ((item.text.lower().startswith(search_words) or item.text.lower() in list_word) or (len([ch for ch in second_effort if ch in item.text]) > 1)) and not check:
            # get_all_information busca o elemento pai que contem todas os pares a ser extraido
            result = get_all_information(item, 0)
            if result[0] == "Found!": # controle para caso confirme a busca do pai
                content = result[1]
                break
            check=True
            
    if content != None:
        filme = {}
        filme['id'] = i
        ultima_categoria = {"status": False, "value": ""}
        trigger_error = False
        if len(content) == 0:
            if content.text:
                filme = extract_by_breakline(content,i)
        else:
            for x in content: # percorre cada filho, buscando os pares a ser extraido
                info = get_element_text(x).strip()
                if info:
                    categoria = which_top_words_startswith(info)
                    if categoria != None:
                        ultima_categoria['status'] = False # para a atribuição iterativa, começa uma nova categoria
                        info = info[len(categoria):].strip() # apenas a informação
                        if info.startswith(":"):
                            info = info[1:].strip() # apenas a informação, sem ':'
                            if info: # não vazio...
    #                             print("com ':'", categoria, info)
                                filme[remover_acentos(categoria).lower()] = info
                            else: # vazio ERRO
                                # tentativa de correção, essa categoria pode ser dita como vazia, 
                                # mas se houver outra ocorrencia, deve ser redirecionado
                                if trigger_error == False:
    #                                 print("erro trigado...")
                                    trigger_error = True
                                    filme[remover_acentos(categoria).lower()] = ""
                                else:
    #                                 print("erro... redirecionando...")
                                    filme = extract_by_breakline(content,i)
                                    break
                        else:
                            if info: # não vazio...
    #                             print("sem ':'", categoria, info)
                                filme[remover_acentos(categoria).lower()] = info
                            else: # vazio, informação nas proximas linhas
                                ultima_categoria['status'] = True
                                ultima_categoria['value'] = categoria
    #                             print("vazio, informação nas proximas linhas...", ultima_categoria['status'], ultima_categoria['value'])
                    elif ultima_categoria['status'] == True and info not in bad_words:
    #                     print("adicionando info interativamente...", info, ultima_categoria['status'], ultima_categoria['value'])
                        cat = remover_acentos(ultima_categoria['value']).lower()
                        if cat not in filme:
                            filme[cat] = []
                        filme[cat].append(info)
                    elif trigger_error == True:
    #                     print("erro... redirecionando...")
                        filme = extract_by_breakline(content,i)
                        break
                else: # vazio
    #                 print("linha vazia...")
                    if trigger_error == True:
    #                     print("erro... redirecionando...")
                        filme = extract_by_breakline(content,i)
                        break
                    ultima_categoria['status'] = False
        print(filme)

{'id': 1, 'direcao': 'Peyton Reed.', 'interpretes': 'Paul Rudd, Evangeline Lilly, Michael Douglas, Michelle Pfeiffer.', 'genero': 'ação. EUA, 2018.', 'duracao': '118 minutos.'}
{'id': 2, 'direcao': 'Susanna Nicchiarelli.', 'interpretes': 'Tryne Dyrholm, John Gordon Sinclair, Anamaria Marinca, Sandor Funtek.', 'genero': 'musical. Itália, 2017.', 'duracao': '93 minutos.'}
{'id': 3, 'direcao': 'Genndy Tartakovsky.', 'genero': 'comédia. Estados Unidos, 2018.', 'duracao': '97 minutos.'}
{'id': 4, 'direcao': 'Ol Parker.', 'elenco': 'Lily James, Amanda Seyfried, Christine Baranski, Pierce Brosnan.', 'genero': 'musical. Reino Unido, 2018.', 'duracao': '114 minutos.'}
{'id': 5, 'direcao': 'Christopher McQuarrie.', 'interpretes': 'Tom Cruise, Sean Harris, Henry Cavill, Rebecca Ferguson.', 'genero': 'thriller.', 'duracao': '147 minutos.'}
{'id': 6, 'direcao': 'Fernando León de Aranoa.', 'elenco': 'Javier Bardem, Penélope Cruz, Peter Sarsgaard.', 'genero': '‘biopic’. Espanha, 2017.', 'duracao': '1

{'id': 31, 'direcao': ['Wes Anderson'], 'elenco': ['Bryan Cranston', 'Koyu Rankin', 'Edward Norton', 'Scarlett Johansson', 'Bob Balaban', 'Greta Gerwig', 'Akira Takayama', 'Yoko Ono', 'Akira Ito', 'Fisher Stevens', 'Liev Schreiber', 'Courtney B. Vance', 'Roman Coppola', 'Anjelica Huston', 'Ken Watanabe', 'F. Murray Abraham', 'Tilda Swinton', 'Frances McDormand', 'Harvey Keitel', 'Jeff Goldblum', 'Bill Murray'], 'roteiro': ['Wes Anderson'], 'producao': ['Wes Anderson', 'Jeremy Dawson', 'Scott Rudin', 'Steven Rales'], 'fotografia': ['Tristan Oliver'], 'musica': ['Alexandre Desplat'], 'montagem': ['Andrew Weisblum', 'Edward Bursch', 'Ralph Foster'], 'design de producao': ['Adam Stockhausen', 'Paul Harrod']}
{'id': 32, 'direcao': ['Stefano Sollima'], 'elenco': ['Benicio Del Toro', 'Josh Brolin', 'Isabela Moner', 'Catherine Keener', 'Jeffrey Donovan', 'Matthew Modine', 'Shea Whigham', 'Manuel Garcia-Rulfo', 'Elijah Rodriguez', 'Bruno Bichir', 'Raoul Trujillo'], 'roteiro': ['Taylor Sheridan'

{'id': 87, 'direcao': 'Susan Johnson', 'elenco': 'Lana Condor, John Corbett, Noah Centineo mais'}
{'id': 88, 'direcao': 'Marcelo Martinessi', 'genero': 'Drama', 'elenco': 'Ana Brun, Margarita Irun, Ana Ivanova.'}
{'id': 89, 'direcao': 'Ol Parker', 'roteiro': 'Ol Parker', 'genero': 'Musical', 'elenco': 'Pierce Brosnan, Meryl Streep, Dominic Cooper, Lily James, Andy García, Amanda Seyfried, Jeremy Irvine, Colin Firth, Hugh Skinner, Stellan Skarsgård, Josh Dylan, Christine Baranski, Jessica Keenan Wynn, Julie Walters, Alexa Davies, Cher.'}
{'id': 90, 'direcao': 'Edouard Deluc', 'roteiro': 'Edouard Deluc, Etienne Comar, Thomas Lilti, Sarah Kaminsky', 'genero': 'Drama, Biografia', 'elenco': 'Vincent Cassel, Tuheï Adams, Malik Zidi, Pernille Bergendorff'}
{'id': 91, 'direcao': 'Eli Roth', 'roteiro': 'Eric Kripke', 'elenco': 'Jack Black, Cate Blanchett, Owen Vaccaro, Renée Elise Goldsberry, Sunny Suljic e Kyle MacLachlan', 'genero': 'Comédia, Fantasia, Família', 'ano': '2018', 'duracao'

In [104]:
list_word = ["intérpretes", "elenco","diretor","direção"]
search_words = ("intérpretes:", "elenco:","diretor:","direção:")
second_effort = ("Intérpretes:", "Elenco:", "Diretor:", "Direção:", "Roteiro:")
# 61 e 91 funciona com split em '\n'
for i in [1,11,21,31,41,51,61,71,81,91]:#range(1,101):
    tree = etree.parse(root + str(i) + '.html', parser)
    check = False
    print("\n\n",i)
    titles = []
    for item in tree.iter():
        if item.get("class") != None and item.get("class").endswith("title"):
            if item.text != None and item.text.strip() != "":
                titles.append(item.text)
    print(titles)
#         if item.text != None and ((item.text.lower().startswith(search_words) or item.text.lower() in list_word) or (len([ch for ch in second_effort if ch in item.text]) > 1)) and not check:
#             print(i)#,(get_element_text(item), get_element_text(item.getparent()), get_element_text(item.getparent().getparent())))
#             result = get_all_information(item, 0)
#             if result[0] == "Found!":
# #                 print(i, get_all_information(item, 0))
# #                 print(get_element_text(result[1]))
# #                 print(len(result[1]))
#                 for x in result[1]:
#                     print(get_element_text(x))
#             print("\n\n**outro site**\n\n")
#             check=True
# #         elif item.text != None and len([ch for ch in second_effort if ch in item.text]) > 1 and not check:
# #             print(get_element_text(item))



 1
[]


 11
['Créditos da imagem:', 'Hotel Artemis', 'Bom de premissa e ruim de entrega, Drew Pearce estreia como diretor com distopia panorâmica', 'Hotel Artemis', 'Hotel Artemis', '\n      Nota do Crítico\n  ']


 21
['O Predador']


 31
[]


 41
[]


 51
['Compartilhe', 'Saiba mais sobre', 'O Predador  (2018)', 'Compartilhe']


 61
[]


 71
['Bloqueio', 'Compartilhe isso:']


 81
['O Retorno do Herói | Crítica', 'Pros', 'Cons', 'Deixe uma Mensagem ', 'MENU', 'Mais Notícias']


 91
['Crítica | O Mistério do Relógio na Parede', 'Compartilhe:', 'Curtir isso:', 'Deixe uma resposta ']
